In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['OMP_NUM_THREADS'] = '3'
os.environ['KMP_BLOCKTIME'] = '1'
import numpy as np
import gc
from tqdm.auto import tqdm
from dataset import get_labelled, get_unlabelled, _preload, _unload, unlabelled_generator, rendred_unlabelled_generator
from render_lookups import render_lookups, render_examples, get_rendered_examples
from Commit import CommitFactory
from Model import CommitDiffModelFactory
import tensorflow as tf

In [2]:
##### META PARAMS #####

# If these two change, the data must be re-rendered!
##########################################

#Size / depth / length of each individual context/path
CONTEXT_SIZE = 8

#Total number of contexts to sample per bag
BAG_SIZE = 128

##########################################

#Encoder to use
ENCODER = 0

#Size of fixed length vector to encode to
OUTPUT_SIZE = 512

# Hard limit to filter bags by. Bags which exceed this size won't even be sampled.
# Set to None to turn off
MAX_BAG_SIZE_FILTER = 2048

#Maximum number of labelled commits to train on
MAX_LABELLED_COMMITS = 4096 * 8

#Maximum number of unlabelled commits to train on
MAX_UNLABELLED_COMMITS = 4096 * 8 * 8

#Batch size for unsupervised training. Higher = faster/better but more memory required
SIAM_BATCH_SIZE = 4096
SIAM_BATCHES_PER_UPDATE = 2

In [3]:
Commit = CommitFactory(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)

In [ ]:
render_lookups(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)

In [ ]:
render_examples(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)

In [4]:
output_types = (tf.float32, tf.float32)
output_shapes = ((BAG_SIZE, CONTEXT_SIZE), (BAG_SIZE, CONTEXT_SIZE))

dataset = tf.data.Dataset.from_generator(
    rendred_unlabelled_generator,
    output_types=output_types,
    output_shapes=output_shapes,
    args=(BAG_SIZE, CONTEXT_SIZE, MAX_UNLABELLED_COMMITS, MAX_BAG_SIZE_FILTER)
)

dataset = dataset.batch(SIAM_BATCH_SIZE)

In [5]:
CommitDiffModel = CommitDiffModelFactory(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE, OUTPUT_SIZE=OUTPUT_SIZE)

In [6]:
model = CommitDiffModel(unsupervised_data_size = MAX_UNLABELLED_COMMITS, siam_batch_size = SIAM_BATCH_SIZE, steps_per_update = SIAM_BATCHES_PER_UPDATE)
model.initialize(encoder=12)

In [ ]:
unsupervised_data = model.fit_siam_generator(dataset, epochs=64, verbose=1)

Epoch 1/64
64/64 [==============================] - 158s 2s/step - loss: -0.5487
Epoch 2/64
64/64 [==============================] - 165s 3s/step - loss: -0.6270
Epoch 3/64
64/64 [==============================] - 155s 2s/step - loss: -0.6319
Epoch 4/64
64/64 [==============================] - 154s 2s/step - loss: -0.6312
Epoch 5/64
64/64 [==============================] - 142s 2s/step - loss: -0.6324
Epoch 6/64
64/64 [==============================] - 141s 2s/step - loss: -0.6362
Epoch 7/64
64/64 [==============================] - 142s 2s/step - loss: -0.6377
Epoch 8/64
64/64 [==============================] - 141s 2s/step - loss: -0.6374
Epoch 9/64
64/64 [==============================] - 141s 2s/step - loss: -0.6388
Epoch 10/64
64/64 [==============================] - 150s 2s/step - loss: -0.6404
Epoch 11/64
64/64 [==============================] - 149s 2s/step - loss: -0.6403
Epoch 12/64
64/64 [==============================] - 158s 2s/step - loss: -0.6423
Epoch 13/64
64/64 [======

In [ ]:
# _preload(max_commits = MAX_LABELLED_COMMITS)

In [ ]:
#X_train, X_test, y_train, y_test = get_labelled(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)
X_train, X_test, y_train, y_test = get_rendered_examples()
X_test_1 = [X_test[i] for i in range(len(X_test)) if y_test[i] == 1]
y_test_1 = [y_test[i] for i in range(len(X_test)) if y_test[i] == 1]
X_test_0 = [X_test[i] for i in range(len(X_test)) if y_test[i] == 0]
y_test_0 = [y_test[i] for i in range(len(X_test)) if y_test[i] == 0]

In [ ]:
print("Train set size", len(y_train))
print("Train set split", np.sum(y_train)/len(y_train))
print("Test set size", len(y_test))
print("Test set split", np.sum(y_test)/len(y_test))

In [ ]:
#For memory optimization
#_unload()

In [ ]:
supervised_results = model.fit_binary_classification(X_train, np.array(y_train), epochs=32, batch_size=128, verbose=1, validation_data=[X_test, np.array(y_test)])

In [ ]:
score = model.evaluate_binary_classification(X_test, np.array(y_test), verbose=0)
print("Score:", score)
score1 = model.evaluate_binary_classification(X_test_1, np.array(y_test_1), verbose=0)
print("Score1:", score1)
score0 = model.evaluate_binary_classification(X_test_0, np.array(y_test_0), verbose=0)
print("Score0:", score0)